In [1]:
pip install xsuite

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%matplotlib widget
import ipympl

import numpy as np
import matplotlib.pyplot as plt

import xpart as xp
import xtrack as xt

fname_model = './generate_models/sps_acceleration.json'

nemitt_x = 2.5e-6
nemitt_y = 2.5e-6

line = xt.Line.from_json(fname_model)
line.build_tracker()

Done loading line from dict.           
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


## Exploring the SPS

The beamline of the SPS is accessible through `line`. Some examples on how we can access the elements of the line.

### Enumerate elements and their names

In [3]:
line.element_names  # Element names

('sps$start',
 'mystart',
 'drift_0',
 'bmo.51997_aper',
 'bmo.51997',
 'drift_1',
 'tacw.51998',
 'drift_2',
 'bpmb.51999_aper',
 'bpmb.51999',
 'drift_3',
 'loen.52002_aper',
 'loen.52002',
 'drift_4',
 'lsf.52005_aper',
 'lsf.52005',
 'drift_5',
 'mdh.52007_aper',
 'mdh.52007',
 'drift_6',
 'bph.52008_aper',
 'bph.52008',
 'drift_7',
 'qf.52010_aper',
 'qf.52010',
 'drift_8',
 'mba.52030_den',
 'drift_9',
 'mba.52030_aper',
 'mba.52030',
 'drift_10',
 'mba.52030_dex',
 'drift_11',
 'mba.52050_den',
 'drift_12',
 'mba.52050_aper',
 'mba.52050',
 'drift_13',
 'mba.52050_dex',
 'drift_14',
 'mbb.52070_den',
 'drift_15',
 'mbb.52070_aper',
 'mbb.52070',
 'drift_16',
 'mbb.52070_dex',
 'drift_17',
 'mbb.52090_den',
 'drift_18',
 'mbb.52090_aper',
 'mbb.52090',
 'drift_19',
 'mbb.52090_dex',
 'drift_20',
 'brcz.52102_aper',
 'brcz.52102',
 'drift_21',
 'mdv.52107_aper',
 'mdv.52107',
 'drift_22',
 'bpv.52108_aper',
 'bpv.52108',
 'drift_23',
 'qd.52110_aper',
 'qd.52110',
 'drift_24',
 'm

In [4]:
line.elements  # Element objects

(<xtrack.beam_elements.elements.Marker at 0x7f2e5b925880>,
 ...)

In [5]:
line['mbb.52070']  # A bending magnet

### Access the properties of individual elements

In [6]:
line['mbb.52070'].to_dict()  # We can view its properties

{'__class__': 'Multipole',
 'order': 0,
 'inv_factorial_order': 1.0,
 'length': 6.26001860278045,
 'hxl': 0.008445141542,
 'hyl': 0.0,
 'radiation_flag': 0,
 'knl': array([0.00844514]),
 'ksl': array([0.])}

In [7]:
# A quadrupole
line['qf.52010']
line['qf.52010'].to_dict()

{'__class__': 'Multipole',
 'order': 1,
 'inv_factorial_order': 1.0,
 'length': 3.085,
 'hxl': 0.0,
 'hyl': 0.0,
 'radiation_flag': 0,
 'knl': array([0.       , 0.0345839]),
 'ksl': array([0., 0.])}

### Twiss

Compute orbit, optics and other quantities of interest.

In [8]:
# Compute the twiss for the beamline
line.particle_ref = xp.Particles(mass0=xp.PROTON_MASS_EV, p0c=26e9)
tw = line.twiss()

In [21]:
fig, (spbet, spdx) = plt.subplots(nrows=2, ncols=1, sharex=True)
spbet.plot(tw.s, tw.betx)
spbet.plot(tw.s, tw.bety)
spbet.set_ylim(bottom=0)
spdx.plot(tw.s, tw.dx)
spdx.plot(tw.s, tw.dy)

spbet.set_ylabel(r'$\beta_{x,y}$ [m]')
spdx.set_ylabel(r"$D_{x,y}$")


fig.suptitle(
    r'$q_x$ = ' f'{tw["qx"]:.5f}' r' $q_y$ = ' f'{tw["qy"]:.5f}' '\n'
    r"$Q'_x$ = " f'{tw["dqx"]:.2f}' r" $Q'_y$ = " f'{tw["dqy"]:.2f}'
    r' $\gamma_{tr}$ = '  f'{1/np.sqrt(tw["momentum_compaction_factor"]):.2f}'
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, "$q_x$ = 20.15000 $q_y$ = 20.25000\n$Q'_x$ = -1.32 $Q'_y$ = -0.72 $\\gamma_{tr}$ = 18.01")

### Measure the tune

In [13]:
num_turns = 500

p_co = tw.particle_on_co.copy()

line.track(p_co, num_turns=num_turns, turn_by_turn_monitor=True)
mon_co = line.record_last_track

p1 = tw.particle_on_co.copy()
p1.x += 0.1e-3
p1.y += 0.2e-3
line.track(p1, num_turns=num_turns, turn_by_turn_monitor=True)
mon1 = line.record_last_track

In [14]:
# Plot turn-by-turn data
fig, (spx, spy) = plt.subplots(nrows=2, ncols=1, sharex=True)
fig.suptitle('Turn-by-turn data')
spx.plot(mon_co.x.T)
spx.plot(mon1.x.T)
spy.plot(mon_co.y.T)
spy.plot(mon1.y.T)
spy.set_xlabel('Turn')
spx.set_ylabel('x [m]')
spy.set_ylabel('y [m]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'y [m]')

In [15]:
# Plot the transverse spectrum
fig, (spx, spy) = plt.subplots(nrows=2, ncols=1, sharex=True)
fig.suptitle('Transverse spectrum')
freq_axis = np.fft.rfftfreq(n=num_turns)
spx.plot(freq_axis, np.abs(np.fft.rfft(mon1.x[0, :])))
spy.plot(freq_axis, np.abs(np.fft.rfft(mon1.y[0, :])))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Consistent with the twiss

In [16]:
tw.qx, tw.qy

(20.15000024623154, 20.25000000092244)

### Survey

In [17]:
# Run the survey
sv = line.survey(element0=0)

In [18]:
# See the whole ring
fig, ax = plt.subplots()
fig.suptitle('SPS Ring')
ax.set_box_aspect(1)
ax.plot(sv.Z, sv.X, label='x', linestyle='--', color='b')

ax.plot(sv.Z, sv.X + tw.x, label='x', linestyle='-', color='b')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
np.trapz(tw.betx, tw.s)/tw.circumference  # measure the simulated average beta function

62.5702959144828

In [20]:
tw.circumference/2/np.pi/tw.qx  # simplified estimate

54.590569753176105